In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="MIG-ffdee303-0dd4-513d-b18c-beba028b49c7" 
from hydra.utils import instantiate
import yaml
import torch
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
device = "cuda:0"

# Load data and models

In [2]:
os.chdir('/allen/aics/modeling/ritvik/projects/benchmarking_representations/')

In [3]:
from br.data.get_datamodules import get_data
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint
from br.models.load_models import load_model_from_path
from br.models.save_embeddings import get_pc_loss

def get_data_and_models(dataset_name, batch_size, results_path, debug=False):
    data_list = get_data(dataset_name, batch_size, results_path, debug)
    all_models, run_names, model_sizes = load_model_from_path(dataset_name, results_path) # default list of models in load_models.py
    return data_list, all_models, run_names, model_sizes

In [6]:
dataset_name = 'pcna'
batch_size = 2
debug=True
results_path = '/allen/aics/modeling/ritvik/projects/benchmarking_representations/br/configs/results/'
data_list, all_models, run_names, model_sizes = get_data_and_models(dataset_name, batch_size, results_path, debug)

In [7]:
all_models

[PointCloudVAE(
   (train/loss): MeanMetric()
   (val/loss): MeanMetric()
   (test/loss): MeanMetric()
   (train/loss/total_reconstruction): MeanMetric()
   (val/loss/total_reconstruction): MeanMetric()
   (test/loss/total_reconstruction): MeanMetric()
   (train/loss/total_kld): MeanMetric()
   (val/loss/total_kld): MeanMetric()
   (test/loss/total_kld): MeanMetric()
   (train/loss/kld_pcloud): MeanMetric()
   (val/loss/kld_pcloud): MeanMetric()
   (test/loss/kld_pcloud): MeanMetric()
   (train/loss/reconstruction_pcloud): MeanMetric()
   (val/loss/reconstruction_pcloud): MeanMetric()
   (test/loss/reconstruction_pcloud): MeanMetric()
   (prior): ModuleDict(
     (pcloud): IdentityPrior()
   )
   (encoder): ModuleDict(
     (pcloud): DGCNN(
       (embedding_head): Linear(in_features=512, out_features=256, bias=True)
       (convs): ModuleList(
         (0): Sequential(
           (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): BatchNorm2d(64, eps=1e-0